In [0]:
%sql
create database fact_dim_main;

In [0]:
%sql
CREATE OR REPLACE TABLE fact_dim_main.Order (
    OrderID INT,
    OrderDate DATE,
    CustomerID INT,
    CustomerName VARCHAR(100),
    CustomerEmail VARCHAR(100),
    ProductID INT,
    ProductName VARCHAR(100),
    ProductCategory VARCHAR(50),
    RegionID INT,
    RegionName VARCHAR(50),
    Country VARCHAR(50),
    Quantity INT,
    UnitPrice DECIMAL(10,2),
    TotalAmount DECIMAL(10,2)
);


In [0]:
%sql
INSERT INTO  fact_dim_main.Order (OrderID, OrderDate, CustomerID, CustomerName, CustomerEmail, ProductID, ProductName, ProductCategory, RegionID, RegionName, Country, Quantity, UnitPrice, TotalAmount) 
VALUES 
(1, '2024-02-01', 101, 'Alice Johnson', 'alice@example.com', 201, 'Laptop', 'Electronics', 301, 'North America', 'USA', 2, 800.00, 1600.00),
(2, '2024-02-02', 102, 'Bob Smith', 'bob@example.com', 202, 'Smartphone', 'Electronics', 302, 'Europe', 'Germany', 1, 500.00, 500.00),
(3, '2024-02-03', 103, 'Charlie Brown', 'charlie@example.com', 203, 'Tablet', 'Electronics', 303, 'Asia', 'India', 3, 300.00, 900.00),
(4, '2024-02-04', 101, 'Alice Johnson', 'alice@example.com', 204, 'Headphones', 'Accessories', 301, 'North America', 'USA', 1, 150.00, 150.00),
(5, '2024-02-05', 104, 'David Lee', 'david@example.com', 205, 'Gaming Console', 'Electronics', 302, 'Europe', 'France', 1, 400.00, 400.00),
(6, '2024-02-06', 102, 'Bob Smith', 'bob@example.com', 206, 'Smartwatch', 'Electronics', 303, 'Asia', 'China', 2, 200.00, 400.00),
(7, '2024-02-07', 105, 'Eve Adams', 'eve@example.com', 201, 'Laptop', 'Electronics', 301, 'North America', 'Canada', 1, 800.00, 800.00),
(8, '2024-02-08', 106, 'Frank Miller', 'frank@example.com', 207, 'Monitor', 'Accessories', 302, 'Europe', 'Italy', 2, 250.00, 500.00),
(9, '2024-02-09', 107, 'Grace White', 'grace@example.com', 208, 'Keyboard', 'Accessories', 303, 'Asia', 'Japan', 3, 100.00, 300.00),
(10, '2024-02-10', 104, 'David Lee', 'david@example.com', 209, 'Mouse', 'Accessories', 301, 'North America', 'USA', 1, 50.00, 50.00);


In [0]:
%sql
SELECT * FROM  fact_dim_main.Order

###Staging

In [0]:
%sql
create database sales_dwh_fact_dim

In [0]:
%sql
create or replace table sales_dwh_fact_dim.orders
as
select * from fact_dim_main.Order

In [0]:
%sql
select * from sales_dwh_fact_dim.orders

###Transform

In [0]:
%sql
create or replace view  sales_dwh_fact_dim.transf_order
as
select * from sales_dwh_fact_dim.orders
where orders.Quantity is not null

###Core Layer

###Dimension Table

###Customer dim

In [0]:
%sql
CREATE OR REPLACE TABLE sales_dwh_fact_dim.customer_dim_table1 (
    CustomerId INT,
    CustomerName STRING,
    CustomerEmail STRING,
    DimCustomerDimensionSurogatekey INT
    
);

In [0]:
%sql
create or replace view  sales_dwh_fact_dim.customer_dimen
as
select T.*,row_number() over(order by T.CustomerId) as DimCustomerDimensionSurogatekey from
( 
select 
 distinct(CustomerId) as CustomerId,
 CustomerName,
 CustomerEmail
from sales_dwh_fact_dim.transf_order
) as T;


In [0]:
%sql
select * from sales_dwh_fact_dim.customer_dimen

In [0]:
%sql
insert into sales_dwh_fact_dim.customer_dim_table1
select * from sales_dwh_fact_dim.customer_dimen

In [0]:
%sql
select * from sales_dwh_fact_dim.customer_dim_table1

###Product dimension

In [0]:
%sql
select * from sales_dwh_fact_dim.orders

In [0]:
%sql
CREATE OR REPLACE TABLE sales_dwh_fact_dim.Product_dimt_table (
    ProductId INT,
    ProductName STRING,
    ProductCategory STRING,
    DimProductDimensionSurogatekey INT
    
);

In [0]:
%sql
create or replace view  sales_dwh_fact_dim.Product_dimen
as
select T.*,row_number() over(order by T.ProductID) as DimProductDimensionSurogatekey from
( 
select 
 distinct(ProductID) as ProductID,
 ProductName,
 ProductCategory 
from sales_dwh_fact_dim.transf_order
) as T;

In [0]:
%sql
select * from sales_dwh_fact_dim.Product_dimen;

In [0]:
%sql
insert into sales_dwh_fact_dim.Product_dimt_table 
select * from sales_dwh_fact_dim.Product_dimen

In [0]:
%sql
select * from sales_dwh_fact_dim.Product_dim_table;

###Region dimension

In [0]:
%sql
select * from sales_dwh_fact_dim.orders

In [0]:
%sql
CREATE OR REPLACE TABLE sales_dwh_fact_dim.Region_dim_table (
    RegionId INT,
    RegionName STRING,
    Country STRING,
    DimRegionDimensionSurogatekey INT
);

In [0]:
%sql
create or replace view  sales_dwh_fact_dim.Region_dimen
as
select T.*,row_number() over(order by T.RegionID) as DimRegionDimensionSurogatekey from
( 
select 
 distinct(RegionID) as RegionID,
 RegionName,
 Country
from sales_dwh_fact_dim.transf_order
) as T;

In [0]:
%sql
insert into  sales_dwh_fact_dim.Region_dim_table
select * from sales_dwh_fact_dim.Region_dimen

In [0]:
%sql
select * from sales_dwh_fact_dim.Region_dim_table

###Date dimension

In [0]:
%sql
select * from sales_dwh_fact_dim.orders

In [0]:
%sql
CREATE OR REPLACE TABLE sales_dwh_fact_dim.Date_dim_table (
    OrderDate DATE,
    DimDateDimensionSurogatekey INT
);

In [0]:
%sql
create or replace view  sales_dwh_fact_dim.Date_dimen
as
select T.*,row_number() over(order by T.OrderDate) as DimDateDimensionSurogatekey from
( 
select 
 distinct(OrderDate) as OrderDate
 from sales_dwh_fact_dim.transf_order
) as T;

In [0]:
%sql
insert into  sales_dwh_fact_dim.Date_dim_table
select * from sales_dwh_fact_dim.Date_dimen

In [0]:
%sql
select * from sales_dwh_fact_dim.Date_dim_table

###FACT TABLE

In [0]:
%sql
select * from sales_dwh_fact_dim.orders

In [0]:
%sql
Create or replace table sales_dwh_fact_dim.fact_table_rect_new_kmn
(
  OrderId INT,
  Quantity INT,
  UnitPrice DECIMAL,
  TotalAmount DECIMAL,
  DimCustomerDimensionSurogatekey INT,
  DimProductDimensionSurogatekey INT,
  DimRegionDimensionSurogatekey INT,
  DimDateDimensionSurogatekey INT
 
)

In [0]:
%sql
create or replace view sales_dwh_fact_dim.temp_fact
as
select
 F.OrderId,
 F.Quantity,
 F.UnitPrice,
 F.TotalAmount,
 DC.DimCustomerDimensionSurogatekey,
 DP.DimProductDimensionSurogatekey,
 DR.DimRegionDimensionSurogatekey,
 DD.DimDateDimensionSurogatekey
 
from sales_dwh_fact_dim.orders F

left join
  sales_dwh_fact_dim.Customer_dim_table1 DC
on
  F.CustomerId = DC.CustomerId

left join
  sales_dwh_fact_dim.Product_dimt_table DP
on
  F.ProductId = DP.ProductId

left join
  sales_dwh_fact_dim.Region_dim_table DR
on  
  F.Country = DR.Country

left join
  sales_dwh_fact_dim.Date_dim_table DD
on
  F.OrderDate = DD.OrderDate

In [0]:
%sql
insert into  sales_dwh_fact_dim.fact_table_rect_new_kmn
select * from sales_dwh_fact_dim.temp_fact
    

In [0]:
%sql
 select * from sales_dwh_fact_dim.fact_table_rect_new_kmn